In [1]:
import Pkg;

In [2]:
using CSV;
using DataFrames;
using LinearAlgebra;

In [3]:
# Import vehicle data as dataframe
dataframe = DataFrame(CSV.File("vehicle.csv"));
println("Dataframe size: ", size(dataframe))

# Number of classes to identify
num_classes = size(unique(dataframe["Class"]),1)

# Labels as dict
label_dict = Dict([(c, findall(x->x==c,unique(dataframe["Class"]))[1]) for c in unique(dataframe["Class"])])

# Create new column, delete classes
classes = dataframe[:,end]
deletecols!(dataframe, size(dataframe,2))
dataframe[!,"Y"] = zeros(Int8, size(dataframe,1))

for i=1:size(dataframe,1)
    dataframe["Y"][i] = label_dict[classes[i]]
end

# Separate Input data and labels
# Training Data
x_train = convert(Array, dataframe[: ,[x for x in names(dataframe) if x != "Y"]][1:700,:])
y_train_ind = convert(Array, dataframe[:, filter(x -> x == "Y", names(dataframe))][1:700,:])
# Correct the label format
y_train = fill(Float64[], (size(x_train,1),1))

for i=1:size(y_train_ind,1)
    new_y = zeros(Float64, num_classes)
    new_y[y_train_ind[i]] = 1.0
    y_train[i] = new_y
end

# Test Data
x_test = convert(Array, dataframe[: ,[x for x in names(dataframe) if x != "Y"]][700:end,:])
y_test_ind = convert(Array, dataframe[:, filter(x -> x == "Y", names(dataframe))][700:end,:])
# Correct the label format
y_test = fill(Float64[], (size(x_train,1),1))
for i=1:size(y_test_ind,1)
    new_y = zeros(Float64, num_classes)
    new_y[y_train_ind[i]] = 1.0
    y_test[i] = new_y
end

# Normalize Data
x_train = x_train ./ maximum(x_train, dims=1)
x_test  = x_test  ./ maximum(x_test , dims=1)

# Display the data
println("Number of samples for training: ", size(x_train,1))
println("Number of samples for test: ", size(x_test,1))
println("Input size: ", size(x_train,2))
println("Number of unique values(labels): ", size(unique(y_train),1))
println("Classes: ", label_dict)

Dataframe size: (846, 19)


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[3]:4
└ @ Core In[3]:4
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[3]:7
└ @ Core In[3]:7
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = (::getfield(Main, Symbol("##3#5")))(::String) at none:0
└ @ Main ./none:0
┌ Warning: `deletecols!(df::DataFrame, inds)` is deprecated, use `select!(df, Not(inds))` instead.
│   caller = top-level scope at In[3]:13
└ @ Core In[3]:13
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[3]:17
└ @ Core ./In[3]:17


Number of samples for training: 700
Number of samples for test: 147
Input size: 18
Number of unique values(labels): 4
Classes: Dict("opel" => 4,"bus" => 3,"saab" => 2,"van" => 1)


In [4]:
mutable struct Layer
    weights::Union{Missing, Array{Float64,2}}
    biases::Union{Missing, Array{Float64,1}}
    z::Array{Float64,2}
    a::Array{Float64,2}
    activation::String
    
    function Layer(num_perceptrons::Int64, prev_layer_size::Int64, activation::String)
        activation_types = ["relu", "softmax", "sigmoid", "leaky_relu", "none"]
        if (activation != "none") && (activation in activation_types)
            # The matrix has column vectors for each activation of previous layer
            # weights[:,1] is the vector of weights for perceptron 1 
            weights = rand(Float64, (prev_layer_size, num_perceptrons))
        elseif activation == "none"
            weights = missing
        else 
            error("Activation function: ", activation, " is not valid.")
        end
        
        biases = zeros(num_perceptrons)
        # Initialize outputs as zeros
        z = zeros(num_perceptrons, 1)
        a = zeros(num_perceptrons, 1)
        new(weights, biases, z, a, activation)
    end
end

struct DeepNeuralNet
    input_size::Int64
    hidden_layers::Array{Layer,1}
    
    function DeepNeuralNet(input_size::Int64)
        new(input_size, [])
    end
end

In [5]:
# Activation Functions
function relu(z)
    r(z) = z < 0 ? 0 : z
    return r.(z)
end
    
function softmax(z::Array{Float64,2})
    return broadcast(exp, z) ./ sum(broadcast(exp, z))
end

softmax (generic function with 1 method)

In [6]:
function add_layer(net::DeepNeuralNet, layer_size::Int64, activation::String)
    if length(net.hidden_layers) == 0
        new_layer = Layer(layer_size, net.input_size, activation)
    else
        new_layer = Layer(layer_size, size(net.hidden_layers[end].weights, 2), activation)
    end
    push!(net.hidden_layers, new_layer)
end

add_layer (generic function with 1 method)

In [7]:
function feed_forward(net::DeepNeuralNet, x::Union{Array{Float64,1},Array{Int64,1}})::Array{Float64,2}
    num_hidden_layers = length(net.hidden_layers)
    layers = net.hidden_layers
   
    # Make x -> 2 Dimensional
    x = reshape(x,(size(x,1),1))
    
    # All layers from first hidden to output layer
    for layer_index=1:num_hidden_layers
        activation_func = layers[layer_index].activation
        weights = layers[layer_index].weights
        biases  = layers[layer_index].biases
        
        # Input to layer 1 of the network
        # use the input values else
        # apply the previous layer outputs
        if layer_index == 1
            layers[layer_index].z = (weights' * x) .+ biases
        else 
            prev_layer_a = layers[layer_index-1].a
            layers[layer_index].z = (weights' * prev_layer_a) .+ biases
        end
        
        # Apply activation function 
        if(activation_func == "relu")    
            layers[layer_index].a = broadcast(relu, layers[layer_index].z)
        elseif(activation_func == "softmax")
            layers[layer_index].a = softmax(layers[layer_index].z)
        end
    end
    return net.hidden_layers[end].a
end

feed_forward (generic function with 1 method)

In [19]:
function back_propagation(net::DeepNeuralNet, pred::Array{Float64,1}, label::Array{Float64,1})
    num_layers = size(net.hidden_layers,1)
    
    for i=reverse(1:num_layers)
        
    end
end

back_propagation (generic function with 1 method)

In [20]:
function train(net::DeepNeuralNet, x::Array{Float64,2}, y::Array{Array{Float64,1},2}; epochs::Int64=20)
    num_data = size(x,1)
    cost_per_epoch::Array{Float64,1} = zeros(Float64, epochs)
    
    # Needs to change according to new output labels
    cross_entropy_loss(preds, label) = -sum(label .* log.(preds))
    
    for epoch=1:epochs
        cost::Float64 = 0.0
        # Iterate through the training data
        for i=1:num_data
            # Predict the output for given data
            prediction = feed_forward(net, x[i,:])
            # Calculate cost and add
            cost = cross_entropy_loss(prediction, y[i])
            # Check if output is correct for backprop
            is_correct_pred = argmax(prediction)[1] == argmax(y[i])
            
            # Backprop
            back_propagation(net, prediction[:,1], y[i])
        end
#         cost_for_iters::Float64 = (-1/num_data) * cost
#         cost_per_epoch[epoch] = cost_for_iters
#         println("Epoch: ", epoch, " Cost: ",  cost_for_iters);
    end
end

train (generic function with 1 method)

In [21]:
# Create the model
num_columns_data = size(x_train,2)
num_classes = size(unique(y_train),1)
model = DeepNeuralNet(num_columns_data)

# Create the layers for the model
add_layer(model, 6, "relu");
add_layer(model, 6, "relu");
add_layer(model, num_classes, "softmax"); # Output Layer

train(model, x_train[5:9,:], reshape(y_train[5:9],5,1), epochs=1)

3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
